## Data Preparation

In [24]:
import pandas as pd
import numpy as np
import re

In [2]:
result = pd.read_csv("Output/books_output.csv")
df = pd.DataFrame(result)
df.head(5).sort_values(by="year", ascending=False)

,isbn10,authors,title,categories,lang,publication-date,year,rating-avg
4,1782216952,[6],20 to Knit: Pocket Pets,"[2942, 2948, 2953]",en,2019-06-04,2019.0,4.66
3,1784943460,[5],50 Knitted Dolls,"[2948, 2953]",en,2017-11-14,2017.0,3.66
1,1501143905,[2],100 Deadly Skills: Survival Edition : The SEAL...,"[376, 378, 868, 873, 900, 2771, 3097, 2590]",en,2016-11-16,2016.0,4.01
2,1444762052,"[3, 4]",The 100 Most Pointless Things in the World : A...,"[2978, 2980]",en,2013-05-23,2013.0,3.59
0,141036141,[1],1984,"[334, 335, 341, 2622, 352, 2626, 353, 2627]",en,2008-10-01,2008.0,4.18


### 1. Analizamos el dataset de libros

In [3]:
# Vemos los data types
df.dtypes

isbn10               object
authors              object
title                object
categories           object
lang                 object
publication-date     object
year                float64
rating-avg          float64
dtype: object

In [4]:
# Analizamos valores nulos

for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

isbn10 - 0.0%
authors - 0.0%
title - 0.0%
categories - 0.0%
lang - 5.0%
publication-date - 0.0%
year - 0.0%
rating-avg - 64.0%


In [5]:
null_cols = df.isnull().sum()
null_cols

isbn10                  0
authors                 0
title                   0
categories              0
lang                 2851
publication-date      139
year                  139
rating-avg          35990
dtype: int64

In [6]:
# Eliminamos las columnas sin años ya que es un campo necesario a la hora de filtrar

df = df.dropna(subset=['year'])

In [7]:
# Modificamos la columna de "publication-date" formato fecha y arreglamos la columna "year"

df["publication-date"] =  pd.to_datetime(df["publication-date"])
df["year"] =  df["year"].apply(lambda x: int(x))
df

,isbn10,authors,title,categories,lang,publication-date,year,rating-avg
0,141036141,[1],1984,"[334, 335, 341, 2622, 352, 2626, 353, 2627]",en,2008-10-01,2008,4.18
1,1501143905,[2],100 Deadly Skills: Survival Edition : The SEAL...,"[376, 378, 868, 873, 900, 2771, 3097, 2590]",en,2016-11-16,2016,4.01
2,1444762052,"[3, 4]",The 100 Most Pointless Things in the World : A...,"[2978, 2980]",en,2013-05-23,2013,3.59
3,1784943460,[5],50 Knitted Dolls,"[2948, 2953]",en,2017-11-14,2017,3.66
4,1782216952,[6],20 to Knit: Pocket Pets,"[2942, 2948, 2953]",en,2019-06-04,2019,4.66
...,...,...,...,...,...,...,...,...
56004,648696707,[50461],2020 Biddy Tarot Planner,"[2820, 2825, 2853]",en,2019-11-25,2019,4.75
56005,1472143051,[50460],100 Greatest Cycling Climbs of Italy : A guide...,"[3063, 3100]",en,2019-11-19,2019,NaN
56006,692792880,[50462],40 Winks : A Narcoleptic's Journey Through Sle...,[2841],en,2016-12-16,2016,3.66
56007,193532271,"[50463, 50464]",100 Carols for Choirs,"[177, 3292, 179, 3294, 183, 3298, 199, 3314, 2...",en,1988-01-07,1988,4.46


In [9]:
# Finalmente eliminamos la columna de fecha de publicación
df = df.drop('publication-date', axis=1)

In [10]:
df.head(5)

,isbn10,authors,title,categories,lang,year,rating-avg
0,141036141,[1],1984,"[334, 335, 341, 2622, 352, 2626, 353, 2627]",en,2008,4.18
1,1501143905,[2],100 Deadly Skills: Survival Edition : The SEAL...,"[376, 378, 868, 873, 900, 2771, 3097, 2590]",en,2016,4.01
2,1444762052,"[3, 4]",The 100 Most Pointless Things in the World : A...,"[2978, 2980]",en,2013,3.59
3,1784943460,[5],50 Knitted Dolls,"[2948, 2953]",en,2017,3.66
4,1782216952,[6],20 to Knit: Pocket Pets,"[2942, 2948, 2953]",en,2019,4.66


### 3. Filtrado y limpieza por géneros

In [ ]:
# Utilizamos la función explode para asignar una fila por género literario y quitamos los caracteres especiales

In [14]:
df = df.assign(categories=df.categories.str.split(",")).explode('categories')

spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['categories'] = df['categories'].str.replace(char, '')

,isbn10,authors,title,categories,lang,year,rating-avg
0,141036141,[1],1984,334,en,2008,4.18
0,141036141,[1],1984,335,en,2008,4.18
0,141036141,[1],1984,341,en,2008,4.18
0,141036141,[1],1984,2622,en,2008,4.18
0,141036141,[1],1984,352,en,2008,4.18


In [15]:
# Renombramos la columna de "categories" por "category_id" para posteriomente poder unirla con el dataset de categorias
df = df.rename(columns={'categories':'category_id'})


In [16]:
df.head(5)

,isbn10,authors,title,category_id,lang,year,rating-avg
0,141036141,[1],1984,334,en,2008,4.18
0,141036141,[1],1984,335,en,2008,4.18
0,141036141,[1],1984,341,en,2008,4.18
0,141036141,[1],1984,2622,en,2008,4.18
0,141036141,[1],1984,352,en,2008,4.18


### 3. Analizamos el dataset de categorias

In [18]:
# Analizamos el dataset de categorias
categories = pd.read_csv("Input/categories.csv")
categories.head(5)

,category_id,category_name
0,1998,.Net Programming
1,176,20th Century & Contemporary Classical Music
2,3291,20th Century & Contemporary Classical Music
3,2659,20th Century History: C 1900 To C 2000
4,2661,21st Century History: From C 2000 -


In [19]:
# Pasamos las dos columnas de category_id a int para poder unirlas. En este caso transformamos la columna del dataset de libros
df["category_id"] = df["category_id"].apply(lambda x: int(x))

In [21]:
# Hacemos merge de ambos dataset por category_id para poder filtrar por nombre
result = pd.merge(df, categories, on='category_id')
result.head(5)

,isbn10,authors,title,category_id,lang,year,rating-avg,category_name
0,141036141,[1],1984,334,en,2008,4.18,Contemporary Fiction
1,1641813911,[731],"Clarissa Harlowe : Or, The History of a Young ...",334,en,2020,NaN,Contemporary Fiction
2,1641813997,[732],Nightmare Abbey,334,en,2020,3.49,Contemporary Fiction
3,164181599X,[747],The Mayor of Casterbridge : The Life and Death...,334,en,2020,NaN,Contemporary Fiction
4,8498414261,[820],Tres senderos hacia el lago / Three Paths to t...,334,es,2011,3.89,Contemporary Fiction


In [22]:
# Eliminamos las columnas que no necesitamos (autores y category_id)
drop_cols = ['authors', "category_id"]
df = df.drop(drop_cols, axis =1)

A continuación están los géneros que vamos a utilizar en este ejercicio_

* Classic Books & Novels
* poetry
* drama
* adventure
* romance
* Thrillers
* biographies and autobiographies
* comic
* science fiction
* life science
* for kids
* romance
* Crime
* Religious
* horror
* Historical Fiction
* Personal Development
* Graphic Novels: Manga
* Guidebooks

In [29]:
genres = ["Classic Books & Novels", "poetry", "drama", "adventure",
"romance", "Thrillers", "biographies", "comic", "science fiction", "life science", "for kids", "romance","Crime",
"Religious","horror", "Historical Fiction", "Personal Development", "Graphic Novels: Manga","Guidebooks"
]

In [26]:
# Filtramos el dataset por las celdas que CONTIENEN los generos anterior (de esta forma no nos limitamos a coincidencias exactas)
# Ignoramos mayúsculas/minúsculas

result = result[result['category_name'].str.contains("Classic Books & Novels|poetry|drama|adventure|romance|Thrillers|biographies|comic|science fiction|life science|for kids|romance|Crime|Religious|horror|Historical Fiction|Personal Development|Graphic Novels: Manga|Guidebooks", regex=True,flags= re.IGNORECASE)]
result.head(5)

,isbn10,authors,title,category_id,lang,year,rating-avg,category_name
844,141036141,[1],1984,335,en,2008,4.18,Classic Books & Novels
845,802130127,[23],The 120 Days of Sodom and Other Writings,335,en,1994,3.46,Classic Books & Novels
846,99629607,[23],The 120 Days Of Sodom : And Other Writings,335,en,1995,3.11,Classic Books & Novels
847,751503843,"[506, 507]",84 Charing Cross Road,335,en,1982,4.20,Classic Books & Novels
848,1641813911,[731],"Clarissa Harlowe : Or, The History of a Young ...",335,en,2020,NaN,Classic Books & Novels


In [27]:
# Vemos que hemos reducido considerablemente nuestro dataset
result.shape

(19313, 8)

In [28]:
# Guardamos el dataset que es con el que vamos a trabajar

result.to_csv('Output/books_clean_dataset.csv')